<a href="https://colab.research.google.com/github/torikabutofkg/-_Anki-/blob/main/QB_pdf_ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #ライブラリのインストール
#@markdown [shift]+[Enter]を押して起動してください

!pip install PyMuPDF==1.21.1
!pip install pdfminer.six==20221105
!pip install PyPDF2==3.0.1
!pip install natsort==5.5.0
#正規表現
import re
import io, copy
import glob, os
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox, LTTextLine, LTChar
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage
import PyPDF2
from PyPDF2 import PdfFileWriter, PdfFileReader
import fitz
from typing import Tuple
import os
import sys
import shutil
import csv
import pandas as pd
import time
from natsort import natsorted
from google.colab import files

In [ ]:
#@title #グーグルドライブと連携
#@markdown [shift]+[Enter]を押して起動してください
#@markdown　<br>GogoleDriveとの連携が求められるので、許可してください。

#ドライブ設定
PATH_GMOUNT='/content/gdrive'
PATH_MYDRIVE=PATH_GMOUNT+'/My Drive'

# Check if the mount point directory exists and is empty
import os
import shutil

# Instead of removing the entire mount point, only remove the folder_name directory if it exists
if os.path.exists(os.path.join(PATH_MYDRIVE, folder_name)):
  # If the directory exists, remove its contents
  shutil.rmtree(os.path.join(PATH_MYDRIVE, folder_name))
  print(f"Existing directory '{folder_name}' contents removed.")

#GDriveマウント
from google.colab import drive
drive.mount(PATH_GMOUNT)

Existing directory 'QBCropper' contents removed.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import shutil
import fitz
import re
import pandas as pd

# ディレクトリの設定
folder_name = "QBCropper"
folder_path = f"/content/gdrive/MyDrive/{folder_name}"
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
os.makedirs(folder_path, exist_ok=True)

pdf_path = "/content/1J.pdf"

# PDFファイルの読み込み
doc = fitz.open(pdf_path)

# 画像を保存するフォルダ
image_folder = os.path.join(folder_path, "images")
os.makedirs(image_folder, exist_ok=True)

# テキスト抽出関数（画像含む）
def extract_text_and_images_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text_list = []
    image_info = {}

    for page_num, page in enumerate(doc, start=1):  # 1ページ目を1から数える
        text = page.get_text()
        text_list.append((page_num, text))  # (ページ番号, テキスト) のタプルで保存

        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            # 一時的な画像ファイル名（ページ番号ベース）
            temp_filename = f"{page_num}_{img_index}.png"
            img_path = os.path.join(image_folder, temp_filename)

            with open(img_path, "wb") as img_file:
                img_file.write(image_bytes)

            # ページごとの画像をリストで保持
            if page_num not in image_info:
                image_info[page_num] = []
            image_info[page_num].append(temp_filename)

    doc.close()
    return text_list, image_info

# 問題の抽出（ページ番号対応）
def extract_questions_from_text(text_list, image_info):
    questions = []
    question_data = {}
    question_text = []
    current_page = 1  # 初期ページ番号を設定
    problem_counter = 0  # 問題ごとのカウンター

    for page_num, text in text_list:  # 各ページのテキストとページ番号
        lines = text.splitlines()
        for line in lines:
            line = line.strip()

            # ページ更新
            current_page = page_num

            # 問題IDの検出
            if "問題ID︓" in line:
                if question_text:
                    question_data['問題'] = "\n".join(question_text).strip()
                    question_text = []

                question_id = re.search(r"問題ID︓(\d+)", line).group(1)
                question_data['問題ID'] = question_id
                question_data['ページ'] = current_page  # ページ番号を記録

                # 問題ごとのカウンターをリセット
                problem_counter += 1

                # 画像が存在する場合、新しい問題IDで画像ファイル名を変更
                if current_page in image_info:
                    new_filenames = []
                    for idx, old_filename in enumerate(image_info[current_page]):
                        new_filename = f"{current_page}_{question_id}_{idx}.png"
                        old_path = os.path.join(image_folder, old_filename)
                        new_path = os.path.join(image_folder, new_filename)
                        os.rename(old_path, new_path)  # ファイル名変更
                        new_filenames.append(new_filename)
                    image_info[current_page] = new_filenames  # 画像リストを更新

            # 解答の検出
            elif "解答︓" in line:
                question_data['解答'] = re.search(r"解答︓([a-e])", line).group(1)

                # 問題ID、解答、問題が揃っている場合に保存
                if '問題' in question_data and '問題ID' in question_data and '解答' in question_data:
                    questions.append(question_data)
                    question_data = {}

            # 問題文の収集
            else:
                if line and not any(skip in line for skip in ["©MEDIC MEDIA", "QB オンラインCBT", "最終更新"]):
                    question_text.append(line)

    # 最後の問題を保存
    if question_text and '問題ID' in question_data and '解答' in question_data:
        question_data['問題'] = "\n".join(question_text).strip()
        questions.append(question_data)

    # 画像の追加
    for question in questions:
        page_num = question["ページ"]  # 記録しておいたページ番号を取得
        if page_num in image_info:
            img_filenames = image_info[page_num]  # そのページの画像リスト
            img_tags = " ".join([f'<img src="{img}" />' for img in img_filenames])
            question["画像"] = img_tags
        else:
            question["画像"] = ""

    return questions

# テキストと画像情報を抽出
pdf_text_list, image_info = extract_text_and_images_from_pdf(pdf_path)

# 問題を抽出
questions = extract_questions_from_text(pdf_text_list, image_info)

# データフレームに変換
df = pd.DataFrame(questions)

# CSVファイルに保存
output_file = os.path.join(folder_path, "1J.csv")
df.to_csv(output_file, index=False, encoding="utf-8")

print(f"CSVファイルが保存されました: {output_file}")


CSVファイルが保存されました: /content/gdrive/MyDrive/QBCropper/1J.csv
